<a href="https://colab.research.google.com/github/luizhc06/ProjetoGeminiAlura/blob/main/GeminiHardware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Partida:***

In [ ]:
!pip install -q -U google-generativeai


In [ ]:
import google.generativeai as genai
import requests
from bs4 import BeautifulSoup

***Geração***

In [ ]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

***Segurança:***

In [ ]:
def recomendar_componentes_usando_IA(componentes_encontrados):
    candidate_count = 3  # Quantidade de candidatos a serem gerados pela IA
    generation_config = {
        "candidate_count": candidate_count,
        "temperature": 0.5,
    }

    safety_settings = {
        "HARASSMENT": "BLOCK_SOME",
        "HATE": "BLOCK_SOME",
        "SEXUAL": "BLOCK_SOME",
        "DANGEROUS": "BLOCK_SOME",
    }

    model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                   generation_config=generation_config,
                                   safety_settings=safety_settings)

    recomendações = []
    for componente in componentes_encontrados:
        prompt = f"Recomende uma opção de {componente['nome']} dentro do orçamento. Usar para {uso}."

        try:
            response = model.generate(prompt)
            recomendações.append({'nome': componente['nome'], 'recomendação': response.candidates[0].text})
        except Exception as e:
            print(f"Erro ao gerar recomendação para {componente['nome']}: {str(e)}")

    return recomendações


In [ ]:
model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                             generation_config=generation_config)

In [ ]:
import googleapiclient.discovery

def pesquisar_no_google(query, api_key, cx):
    """
    Realiza uma pesquisa no Google usando a API Custom Search e retorna os resultados.

    Argumentos:
        query (str): Termo de pesquisa.
        api_key (str): Chave de API do Google.
        cx (str): ID da Pesquisa Customizada.

    Retorno:
        Lista de dicionários contendo informações sobre os resultados da pesquisa.
    """

    service = googleapiclient.discovery.build('customsearch', 'v1', developerKey=api_key)
    try:
        resultados = service.cse().list(q=query, cx=cx).execute()
        return resultados.get('items', [])
    except Exception as e:
        print(f"Erro ao realizar a pesquisa no Google: {str(e)}")
        return []

# Configurações da API de Pesquisa Customizada do Google
api_key = ''  # Substitua com sua chave de API válida
cx = ''  # Substitua com seu ID de Pesquisa Customizada

def capturar_preferencias():
    """
    Captura as preferências do usuário (orçamento, uso, marcas preferidas).

    Retorno:
        Tuple contendo (orçamento, uso, preferencia_processador, preferencia_placa_video).
    """

    orçamento = float(input("Qual é o seu orçamento máximo? R$ "))
    uso = input("Para que você vai usar o computador? ").lower()

    while True:
        preferencia_processador = input("Qual marca de processador você prefere? (AMD, Intel, Nenhuma) ").lower()
        if preferencia_processador in ['amd', 'intel', 'nenhuma']:
            break
        else:
            print("Opção inválida. Por favor, escolha entre AMD, Intel ou Nenhuma.")

    while True:
        preferencia_placa_video = input("Qual marca de placa de vídeo você prefere? (AMD, Intel, Nvidia, Nenhuma) ").lower()
        if preferencia_placa_video in ['amd', 'intel', 'nvidia', 'nenhuma']:
            break
        else:
            print("Opção inválida. Por favor, escolha entre AMD, Intel, Nvidia ou Nenhuma.")

    return orçamento, uso, preferencia_processador, preferencia_placa_video

# Capturar preferências do usuário
orçamento, uso, pref_processador, pref_placa_video = capturar_preferencias()

# Realizar uma pesquisa no Google com base nas preferências do usuário
query = f"{pref_processador} {pref_placa_video} gaming components"
resultados_google = pesquisar_no_google(query, api_key, cx)

# Processar e exibir os resultados da pesquisa do Google
if resultados_google:
    for item in resultados_google:
        nome = item['title']
        link = item['link']
        print(f"Nome: {nome}")
        print(f"Link: {link}")
        print("-" * 30)
else:
    print("Nenhum resultado encontrado na pesquisa do Google.")
